In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score

## Pull CSV Files

In [3]:
zori = pd.read_csv('./data/cleandata/clean_zori.csv').drop('Unnamed: 0', axis = 1)
zhvi = pd.read_csv('./data/cleandata/clean_zhvi.csv').drop('Unnamed: 0', axis = 1)
airq = pd.read_csv('./data/cleandata/clean_airq.csv').drop('Unnamed: 0', axis = 1)
population = pd.read_csv('./data/cleandata/clean_population.csv').drop('Unnamed: 0', axis = 1)
unemployment = pd.read_csv('./data/cleandata/clean_unemployment.csv').drop('Unnamed: 0', axis = 1)
education = pd.read_csv('./data/cleandata/clean_education.csv').drop('Unnamed: 0', axis = 1)
permits = pd.read_csv('./data/cleandata/clean_permits.csv').drop('Unnamed: 0', axis = 1)
IandH = pd.read_csv('./data/cleandata/clean_IandH.csv').drop('Unnamed: 0', axis = 1)
pce = pd.read_csv('./data/cleandata/clean_pce.csv').drop('Unnamed: 0', axis = 1)
vacancy = pd.read_csv('./data/cleandata/clean_vacancy.csv').drop('Unnamed: 0', axis = 1)

## Intermediate Matrices for Merging Dataframes before Training

In [4]:
one_year_forecast = zori[['Year', 'Month', 'Year_Month']]
one_year_forecast = pd.concat([one_year_forecast, pd.DataFrame(one_year_forecast['Year'].map(lambda year: year - 1))], 
                             axis = 1)
one_year_forecast.columns = ['Year', 'Month', 'Year_Month', 'Year2']
one_year_forecast = pd.concat([one_year_forecast, pd.DataFrame(one_year_forecast['Year'].map(lambda year: year - 2))], 
                             axis = 1)
one_year_forecast.columns = ['Year', 'Month', 'Year_Month', 'Year2', 'Year3']
one_year_forecast['Year_Month2'] = one_year_forecast['Year2'].map(str) + '_' + one_year_forecast['Month'].map(str)
one_year_forecast['Year_Month3'] = one_year_forecast['Year3'].map(str) + '_' + one_year_forecast['Month'].map(str)
one_year_forecast = one_year_forecast[['Year_Month', 'Year_Month2', 'Year_Month3']]
one_year_forecast.drop_duplicates(inplace = True)

In [5]:
one_year_forecast

,Year_Month,Year_Month2,Year_Month3
0,2014_2,2013_2,2012_2
1,2014_3,2013_3,2012_3
2,2014_4,2013_4,2012_4
3,2014_5,2013_5,2012_5
4,2014_6,2013_6,2012_6
...,...,...,...
75,2020_5,2019_5,2018_5
76,2020_6,2019_6,2018_6
77,2020_7,2019_7,2018_7
78,2020_8,2019_8,2018_8


## Final Dataframe for Training

In [147]:
df = zori
df = pd.merge(df, one_year_forecast, on = 'Year_Month')
df = pd.merge(df, zori[['Year_Month', 'ZipCode', 'ZORI']], left_on = ['Year_Month2', 'ZipCode'],
              right_on = ['Year_Month', 'ZipCode'], how = 'left')
df.drop('Year_Month_y', axis = 1, inplace = True)

In [148]:
df = pd.merge(df, zhvi[['Year_Month', 'ZipCode', 'ZHVI']], left_on = ['Year_Month2', 'ZipCode'],
              right_on = ['Year_Month', 'ZipCode'], how = 'left')
df.drop('Year_Month', axis = 1, inplace = True)

In [149]:
df = pd.merge(df, airq[['County', 'State', 'Year_Month', 'AQI']], left_on = ['County', 'State', 'Year_Month2'],
              right_on = ['County', 'State', 'Year_Month'], how = 'left')
df.drop('Year_Month', axis = 1, inplace = True)

In [150]:
df = pd.merge(df, population[['County', 'State', 'Year_Month', 'Population']], 
              left_on = ['County', 'State', 'Year_Month3'], right_on = ['County', 'State', 'Year_Month'], how = 'left')
df.drop('Year_Month', axis = 1, inplace = True)

In [151]:
df = pd.merge(df, unemployment[['County', 'State', 'Year_Month', 'Unemployment']], 
             left_on = ['County', 'State', 'Year_Month2'], right_on = ['County', 'State', 'Year_Month'], how = 'left')
df.drop('Year_Month', axis = 1, inplace = True)

In [152]:
df['Year2'] = df['Year'] - 1
df['Year3'] = df['Year'] - 2
df = pd.merge(df, education, left_on = ['County', 'State', 'Year3'], right_on = ['County', 'State', 'Year'], how = 'left')
df.drop('Year_y', axis = 1, inplace = True)

In [153]:
df = pd.merge(df, permits[['State', 'Units', 'Year_Month']], left_on = ['State', 'Year_Month2'], 
              right_on = ['State', 'Year_Month'], how = 'left')
df.drop('Year_Month', axis = 1, inplace = True)

In [154]:
df = pd.merge(df, IandH[['County', 'State', 'Year_Month', 'Total_Households', 'Med_income']], 
             left_on = ['County', 'State', 'Year_Month3'], right_on = ['County', 'State', 'Year_Month'], how = 'left')
df.drop('Year_Month', axis = 1, inplace = True)

In [155]:
df = pd.merge(df, pce[['PCE', 'Year_Month']], left_on = 'Year_Month2', right_on = 'Year_Month')
df.drop('Year_Month', axis = 1, inplace = True)

In [156]:
df = pd.merge(df, vacancy, left_on = ['County', 'State', 'Year3'], right_on = ['County', 'State', 'Year'], how = 'left')
df.drop('Year', axis = 1, inplace = True)

In [157]:
df = df[['Year_x', 'Month', 'ZipCode', 'ZORI_x', 'ZORI_y', 'ZHVI', 'AQI', 'Population', 'Unemployment', 
         'Percent Bachelors', 'Units', 'Total_Households', 'Med_income', 'PCE', 'Rental Vacancy Rate']]

In [158]:
df.columns = ['Year', 'Month', 'ZipCode', 'ZORI', 'ZORI_lagged_1', 'ZHVI_lagged_1', 'AQI_lagged_1', 'Population_lagged_2', 
              'Unemployment_lagged_1', 'Percent Bachelors_lagged_2', 'Permits_lagged_1', 'Total_Households_lagged_2',
              'Med_Income_lagged_2', 'PCE_lagged_1', 'Rental Vacancy Rate_lagged_2']

In [159]:
# Dropping all NaN values instead of imputing for simplicity for now
df = df[(df['Year'] != 2014) & ~((df['Year'] == 2015) & (df['Month'] == 1))]
df = df[~df['Total_Households_lagged_2'].isnull()]
df = df[~df['ZORI'].isnull()]
df = df[~df['ZORI_lagged_1'].isnull()]
df = df[~df['ZHVI_lagged_1'].isnull()]
df = df[~df['AQI_lagged_1'].isnull()]
df = df[~df['Rental Vacancy Rate_lagged_2'].isnull()]
train = df[df['Year'] != 2020]
test = df[df['Year'] == 2020]

## Multiple Linear Regression on ZORI

In [160]:
regression = LinearRegression()
regression.fit(train.iloc[:, 4:], train.iloc[:, 3])
regression.score(train.iloc[:, 4:], train.iloc[:, 3])

0.9925193482593228

In [161]:
r2_score(test.iloc[:, 3], regression.predict(test.iloc[:, 4:]))

0.9801955421585727

In [162]:
# Regression to see how much prior year ZORI drives current year ZORI
temp = LinearRegression()
temp.fit(np.array(train.iloc[:, 4]).reshape(-1, 1), train.iloc[:, 3])
temp.score(np.array(train.iloc[:, 4]).reshape(-1, 1), train.iloc[:, 3])

0.9915774934484615

In [163]:
r2_score(test.iloc[:, 3], temp.predict(np.array(test.iloc[:, 4]).reshape(-1, 1)))

0.978426063311627

# Multiple Linear Regression on Change in ZORI

In [222]:
df_change = df.copy()
df_change['Year_Month'] = df_change['Year'].map(str) + '_' + df_change['Month'].map(str)
df_change = pd.merge(df_change, one_year_forecast, on = 'Year_Month')
df_change = pd.merge(df_change, df_change, left_on = ['Year_Month', 'ZipCode'], right_on = ['Year_Month2', 'ZipCode'],
                     how = 'inner')

In [223]:
df_change['ZORI_x'] = df_change['ZORI_x'] - df_change['ZORI_y']
df_change['ZHVI_lagged_1_x'] = df_change['ZHVI_lagged_1_x'] - df_change['ZHVI_lagged_1_y']
df_change['AQI_lagged_1_x'] = df_change['AQI_lagged_1_x'] - df_change['AQI_lagged_1_y']
df_change['Population_lagged_2_x'] = df_change['Population_lagged_2_x'] - df_change['Population_lagged_2_y']
df_change['Unemployment_lagged_1_x'] = df_change['Unemployment_lagged_1_x'] - df_change['Unemployment_lagged_1_y']
df_change['Percent Bacehlors_lagged_2_x'] = df_change['Percent Bachelors_lagged_2_x'] -\
                                            df_change['Percent Bachelors_lagged_2_y']
df_change['Permits_lagged_1_x'] = df_change['Permits_lagged_1_x'] - df_change['Permits_lagged_1_y']
df_change['Total_Households_lagged_2_x'] = df_change['Total_Households_lagged_2_x'] -\
                                           df_change['Total_Households_lagged_2_y']
df_change['Med_Income_lagged_2_x'] = df_change['Med_Income_lagged_2_x'] - df_change['Med_Income_lagged_2_y']
df_change['PCE_lagged_1_x'] = df_change['PCE_lagged_1_x'] - df_change['PCE_lagged_1_y']
df_change['Rental Vacancy Rate_lagged_2_x'] = df_change['Rental Vacancy Rate_lagged_2_x'] -\
                                              df_change['Rental Vacancy Rate_lagged_2_y']

In [224]:
df_change = df_change[['Year_x', 'Month_x', 'ZipCode', 'ZORI_x', 'ZHVI_lagged_1_x', 'AQI_lagged_1_x', 
                      'Population_lagged_2_x', 'Unemployment_lagged_1_x', 'Percent Bachelors_lagged_2_x', 
                      'Permits_lagged_1_x', 'Total_Households_lagged_2_x', 'Med_Income_lagged_2_x', 'PCE_lagged_1_x', 
                      'Rental Vacancy Rate_lagged_2_x']]

In [225]:
df_change.columns = ['Year', 'Month', 'ZipCode', 'ZORI_delta', 'ZHVI_lagged_1_delta', 'AQI_lagged_1_delta',
                    'Population_lagged_2_delta', 'Unemployment_lagged_1_delta', 'Percent Bachelors_lagged_2_delta', 
                    'Permits_lagged_1_delta', 'Total_Households_lagged_2_delta', 'Med_Income_lagged_2_delta', 
                    'PCE_lagged_1_delta', 'Rental Vacancy Rate_lagged_2_delta']

In [226]:
train_change = df_change[df_change['Year'] != 2019]
test_change = df_change[df_change['Year'] == 2019]

In [227]:
regression_change = LinearRegression()
regression.fit(train_change.iloc[:, 4:], train_change.iloc[:, 3])
regression.score(train_change.iloc[:, 4:], train_change.iloc[:, 3])

0.012174841246029522

In [228]:
r2_score(test_change.iloc[:, 3], regression.predict(test_change.iloc[:, 4:]))

0.0007700455758817837